- Annual brick production in Bangladesh in 2018: 33 billion for 7,859 kilns
- https://www.ccacoalition.org/sites/default/files/resources/2019_Report_Bangladesh%20Brick%20Sector%20Roadmap.pdf

In [1]:
import xarray as xr
import os
import pandas as pd
import numpy as np
import dask
import dask.array as da
import netCDF4
import zarr
import gcsfs
import esmpy
import xesmf as xe
import geopandas as gpd
import rioxarray
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import cartopy.crs as ccrs
from shapely.ops import transform
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pycountry_convert as pc
import glob
import calendar
import datetime
from shapely import wkt
from shapely.geometry import Point

In [2]:
path = "/Users/akawano/Library/CloudStorage/GoogleDrive-akawano@stanford.edu/My Drive/MyProjects/04_brick_kiln_emissions/emission_data"
dist_path = "/Users/akawano/Library/CloudStorage/GoogleDrive-akawano@stanford.edu/My Drive/MyProjects/04_brick_kiln_emissions"

In [54]:
# Import kiln location data
kilns_gps = gpd.read_file(os.path.join(path, "gps_all_kilns.shp"))
print(kilns_gps.head())

RCT_kilns = kilns_gps[kilns_gps['category']=="RCT"]
print(RCT_kilns.head())

Scaling_kilns = kilns_gps[kilns_gps['category']=="Scaling"]
print(scaling_kilns.head())

PNAS_kilns = kilns_gps[kilns_gps['category']=="PNAS"]
PNAS_kilns.head()

   kiln_id category    division                         geometry
0   100000     PNAS  Chittagong  POINT (5241276.392 3739925.248)
1   100001     PNAS  Chittagong  POINT (5241189.938 3740082.634)
2   100002     PNAS  Chittagong  POINT (5240070.054 3740287.121)
3   100003     PNAS  Chittagong  POINT (5241021.955 3740369.603)
4   100004     PNAS  Chittagong  POINT (5239764.288 3751000.298)
      kiln_id category division                         geometry
6129       17      RCT   Khulna  POINT (4919851.782 3911492.283)
6130       16      RCT   Khulna  POINT (4925799.527 3922897.878)
6131        3      RCT   Khulna  POINT (4896473.825 3925735.083)
6132        7      RCT   Khulna  POINT (4932952.327 3932790.311)
6133       23      RCT   Khulna  POINT (4939397.309 3940380.672)
      kiln_id category division                         geometry
6719    26454  Scaling    Dhaka   POINT (5043743.099 3998466.13)
6720    26453  Scaling    Dhaka  POINT (5042402.286 3998463.656)
6721    26452  Scaling   

,kiln_id,category,division,geometry
0,100000,PNAS,Chittagong,POINT (5241276.392 3739925.248)
1,100001,PNAS,Chittagong,POINT (5241189.938 3740082.634)
2,100002,PNAS,Chittagong,POINT (5240070.054 3740287.121)
3,100003,PNAS,Chittagong,POINT (5241021.955 3740369.603)
4,100004,PNAS,Chittagong,POINT (5239764.288 3751000.298)


In [7]:
# Read RCT emissions data
# Emissions data for RCT kilns
# CO2 emissions are reported in tons/100,000 bricks
# PM2.5 emissions are kg/100,000 bricks

# CO2 and PM2.5 emissions multiplied by annual brick production (variable “annual_production_actual_lakh_bricks”), 
# a variable we only have for RCT kilns =  season-wide CO2 emissions in tons and PM2.5 emissions in kg.

emi = pd.read_csv(os.path.join(path, "rct_emissions_data.csv"))
emi['CO2_tons_season'] = emi['co2_emission'] * emi['annual_production_actual_lakh_bricks']
emi['pm25_kg_season'] = emi['pm_2_5_emissions'] * emi['annual_production_actual_lakh_bricks']

# get how many months each kiln operated in this firing season
# create firing start date 
month_to_num = {month.lower(): index for index, month in enumerate(calendar.month_name) if month}

# Define which months correspond to each year
months_2022 = ['october', 'november', 'december']
months_2023 = ['january', 'february', 'march', 'april', 'may', 'june']

def get_week_start_date(year, month, week):
    # Get the first day of the month
    first_day = datetime.date(year, month, 1)
    # Find the first Monday of the month (if the first day isn't already a Monday)
    first_monday = first_day + datetime.timedelta(days=(0 - first_day.weekday()) % 7)
    # Calculate the Monday of the desired week
    return first_monday + datetime.timedelta(weeks=week-1)

# Apply the function with a conditional for the year
emi['firing_start_date'] = emi.apply(
    lambda row: get_week_start_date(
        2022 if row['firing_month'].lower() in months_2022 else 2023,
        month_to_num[row['firing_month'].lower()],
        int(row['firing_week'])
    ),
    axis=1
)

# Convert firing_end_date similarly, and fix the misplaced parenthesis
emi['firing_end_date'] = emi.apply(
    lambda rw: get_week_start_date(
        2023, 
        month_to_num[row['firing_end_month'].lower()],  # map month name to number
        int(row['firing_end_week'])
    ),
    axis=1
)
emi['firing_start_date'] = pd.to_datetime(emi['firing_start_date'])
emi['firing_end_date'] = pd.to_datetime(emi['firing_end_date'])

# Calculate season_days as an integer representing the number of days
emi['season_days'] = (emi['firing_end_date'] - emi['firing_start_date']).dt.days

emi['avg_co2_tons_per_day'] = emi['CO2_tons_season']/emi['season_days'] 
emi['avg_co2_tons_per_month'] = emi['avg_co2_tons_per_day']*30 

emi['avg_pm25_kg_per_day'] = emi['pm25_kg_season']/emi['season_days'] 
emi['avg_pm25_kg_per_month'] = emi['avg_pm25_kg_per_day']*30 

emi['category'] = 'RCT'

#get per day emission, then multiply
emi.head()

,kiln_id,kiln_district,treatment,treatment_bundled,firing_end_month,firing_end_week,kiln_bricks_fired_lakhs,kiln_circuits_completed,production_cost_estimate_bdt_per_1k_bricks,annual_production_targeted_lakh_bricks,...,CO2_tons_season,pm25_kg_season,firing_start_date,firing_end_date,season_days,avg_co2_tons_per_day,avg_co2_tons_per_month,avg_pm25_kg_per_day,avg_pm25_kg_per_month,category
0,44053,jhenaidah,incentive,g,may,3,8.22,9.5,9200,90,...,1606.559399,4241.808184,2022-11-14,2023-05-15,182,8.827249,264.817483,23.306638,699.199151,RCT
1,41091,jashore,control,control,july,4,8.40,10.5,9000,90,...,2608.164201,6886.351204,2022-12-12,2023-07-24,224,11.643590,349.307705,30.742639,922.279179,RCT
2,41046,jashore,incentive,g,july,2,8.10,4.5,9800,60,...,985.608992,2602.309190,2022-11-28,2023-07-10,224,4.400040,132.001204,11.617452,348.523552,RCT
3,41075,jashore,control,control,july,2,8.24,4.5,9000,60,...,953.259190,2516.895820,2022-12-19,2023-07-10,203,4.695858,140.875742,12.398502,371.955047,RCT
4,47051,khulna,control,control,july,4,8.20,13.5,8500,80,...,3355.432650,8859.368466,2022-11-14,2023-07-24,252,13.315209,399.456268,35.156224,1054.686722,RCT


In [9]:
# average season_days amongst RCT kilns
operation_avg_days = emi['season_days'].mean()
print(operation_avg_days)

# average annual production brickls
brick_mean = emi['annual_production_actual_lakh_bricks'].mean()
print(brick_mean)

166.6304347826087
62.593492391304345


In [11]:
# benchmark annual brick production in Bangladesh in 2018: 33 billion for 7,859 kilns
# https://www.ccacoalition.org/sites/default/files/resources/2019_Report_Bangladesh%20Brick%20Sector%20Roadmap.pdf

33000000000/7859

4199007.507316452

In [13]:
# take the average of brick mean production in RCT emissions data and public report
brick_mean = (42+63)/2
brick_mean

52.5

In [41]:
## Sub-district boundaries
#subdist = gpd.read_file("/Users/akawano/Library/CloudStorage/GoogleDrive-akawano@stanford.edu/My Drive/MyProjects/04_brick_kiln_emissions/bgd_adm_bbs_20201113_SHP/bgd_admbnda_adm3_bbs_20201113.shp")
#subdist = subdist[subdist['ADM1_EN']=='Dhaka'] #only dhaka
#subdist = subdist[['ADM3_EN', 'ADM2_EN','geometry']].copy()
#subdist['ADM3_EN'] = subdist['ADM3_EN'].str.lower()
#subdist['ADM2_EN'] = subdist['ADM2_EN'].str.lower()
#subdist['centroid'] = subdist['geometry'].centroid
#subdist = subdist.drop(columns = 'geometry').rename(columns = {'centroid':'geometry'})
#subdist["latitude"] = subdist.geometry.map(lambda p: p.y)
#subdist["longitude"] = subdist.geometry.map(lambda p: p.x)
#subdist = subdist.drop(columns = 'geometry')
#subdist = subdist.rename(columns = {'ADM3_EN':'kiln_sub_district', 'ADM2_EN':'kiln_district'})
#subdist.head()


In [43]:
emi2 = pd.read_csv(os.path.join(path, "scaling_emissions_data.csv"))

emi2['CO2_tons_season'] = emi2['co2_emission'] * brick_mean # use average annual production of bricks available in RCT
emi2['pm25_kg_season'] = emi2['pm_2_5_emissions'] * brick_mean

# Convert firing_start_date using the mapped month numbers
emi2['firing_start_date'] = emi2.apply(
    lambda row: get_week_start_date(
        2022, 
        month_to_num[row['firing_season_firing_month'].lower()],  # map month name to number
        int(row['firing_season_firing_week'])
    ),
    axis=1
)

emi2['firing_start_date'] = pd.to_datetime(emi2['firing_start_date'])
emi2['firing_end_date'] = emi2['firing_start_date'] + pd.to_timedelta(operation_avg_days, unit='D') # use data from RCT

# Calculate season_days as an integer representing the number of days
emi2['season_days'] = operation_avg_days

emi2['avg_co2_tons_per_day'] = emi2['CO2_tons_season']/emi2['season_days'] 
emi2['avg_co2_tons_per_month'] = emi2['avg_co2_tons_per_day']*30 

emi2['avg_pm25_kg_per_day'] = emi2['pm25_kg_season']/emi2['season_days'] 
emi2['avg_pm25_kg_per_month'] = emi2['avg_pm25_kg_per_day']*30 

emi2['category'] = 'Scaling'
emi2.head()

,kiln_id,collection_date,stage,kiln_division,kiln_district,kiln_sub_district,firing_season_firing_month,firing_season_firing_week,firing_season_circuits_completed,firing_season_circuits_current,...,CO2_tons_season,pm25_kg_season,firing_start_date,firing_end_date,season_days,avg_co2_tons_per_day,avg_co2_tons_per_month,avg_pm25_kg_per_day,avg_pm25_kg_per_month,category
0,33183,2024-05-09,stage 2 - No Info,dhaka,gazipur,kaliakair,november,2,9,10,...,1521.480843,4017.174775,2022-11-14,2023-04-29 15:07:49.565217391,166.630435,9.130870,273.926101,24.108290,723.248687,Scaling
1,33096,2024-05-12,stage 1,dhaka,gazipur,kaliakair,december,2,9,10,...,1248.641553,3296.795602,2022-12-12,2023-05-27 15:07:49.565217391,166.630435,7.493478,224.804350,19.785075,593.552241,Scaling
2,33051,2024-05-07,stage 2 - No Info,dhaka,gazipur,kaliakair,november,3,6,7,...,1570.380091,4146.283747,2022-11-21,2023-05-06 15:07:49.565217391,166.630435,9.424329,282.729879,24.883112,746.493356,Scaling
3,33032,2024-05-05,stage 2 - Info,dhaka,gazipur,kaliakair,november,1,9,10,...,1693.724993,4471.952014,2022-11-07,2023-04-22 15:07:49.565217391,166.630435,10.164560,304.936789,26.837546,805.126390,Scaling
4,33006,2024-03-28,stage 2 - Info,dhaka,gazipur,kapasia,december,1,6,7,...,1950.092208,5148.839871,2022-12-05,2023-05-20 15:07:49.565217391,166.630435,11.703097,351.092922,30.899757,926.992697,Scaling


## Merge emissions data with kiln locations

In [46]:
emissions = pd.concat([emi, emi_scale])
emissions = emissions[['kiln_id', 'category','firing_start_date','firing_end_date','season_days','avg_co2_tons_per_day','avg_co2_tons_per_month',
'avg_pm25_kg_per_day','avg_pm25_kg_per_month', 'latitude','longitude']].copy()

emissions['firing_end_date'] = emissions['firing_end_date'].dt.strftime('%Y-%m-%d')
emissions.head()

,kiln_id,category,firing_start_date,firing_end_date,season_days,avg_co2_tons_per_day,avg_co2_tons_per_month,avg_pm25_kg_per_day,avg_pm25_kg_per_month,latitude,longitude
0,44053,RCT,2022-11-14,2023-05-15,182.0,8.827249,264.817483,23.306638,699.199151,NaN,NaN
1,41091,RCT,2022-12-12,2023-07-24,224.0,11.643590,349.307705,30.742639,922.279179,NaN,NaN
2,41046,RCT,2022-11-28,2023-07-10,224.0,4.400040,132.001204,11.617452,348.523552,NaN,NaN
3,41075,RCT,2022-12-19,2023-07-10,203.0,4.695858,140.875742,12.398502,371.955047,NaN,NaN
4,47051,RCT,2022-11-14,2023-07-24,252.0,13.315209,399.456268,35.156224,1054.686722,NaN,NaN


In [50]:
# Match with location datae - RCT

emissions_RCT = emissions[emissions['category']=='RCT']
emissions_else = emissions[emissions['category']!='RCT']
emissions_data = pd.merge(emissions_RCT.drop(columns = ['latitude','longitude']), RCT_kilns[['kiln_id']].copy(), on = ['kiln_id'], how = 'left')
emissions_data.head()


,kiln_id,category,firing_start_date,firing_end_date,season_days,avg_co2_tons_per_day,avg_co2_tons_per_month,avg_pm25_kg_per_day,avg_pm25_kg_per_month
0,44053,RCT,2022-11-14,2023-05-15,182.0,8.827249,264.817483,23.306638,699.199151
1,41091,RCT,2022-12-12,2023-07-24,224.0,11.643590,349.307705,30.742639,922.279179
2,41046,RCT,2022-11-28,2023-07-10,224.0,4.400040,132.001204,11.617452,348.523552
3,41075,RCT,2022-12-19,2023-07-10,203.0,4.695858,140.875742,12.398502,371.955047
4,47051,RCT,2022-11-14,2023-07-24,252.0,13.315209,399.456268,35.156224,1054.686722


In [56]:
# Match with location datae - Scaling
emissions_scale = pd.merge(emissions_else.drop(columns = ['latitude','longitude']), Scaling_kilns[['kiln_id']].copy(), on = ['kiln_id'], how = 'left')
emissions_scale.head()

,kiln_id,category,firing_start_date,firing_end_date,season_days,avg_co2_tons_per_day,avg_co2_tons_per_month,avg_pm25_kg_per_day,avg_pm25_kg_per_month
0,33183,Scaling,2022-11-14,2023-04-29,166.630435,9.130870,273.926101,24.108290,723.248687
1,33096,Scaling,2022-12-12,2023-05-27,166.630435,7.493478,224.804350,19.785075,593.552241
2,33051,Scaling,2022-11-21,2023-05-06,166.630435,9.424329,282.729879,24.883112,746.493356
3,33032,Scaling,2022-11-07,2023-04-22,166.630435,10.164560,304.936789,26.837546,805.126390
4,33006,Scaling,2022-12-05,2023-05-20,166.630435,11.703097,351.092922,30.899757,926.992697


In [60]:
# concatenate RCT and scaling emissions
emissions_df = pd.concat([emissions_data, emissions_else])

In [62]:
emissions_df['category'].unique()

array(['RCT', 'Scaling'], dtype=object)

In [70]:
# CO2 and PM2.5 emissions from nearby kilns in RCT & Scaling

geometry = [Point(xy) for xy in zip(emissions_df['longitude'], emissions_df['latitude'])]

# Create a GeoDataFrame using the geometry column
emissions_gdf = gpd.GeoDataFrame(emissions_df, geometry=geometry, crs = 4326)
emissions_gdf = emissions_gdf.to_crs(7755)

# 1. Ensure both GeoDataFrames have the same CRS
location_gdf = PNAS_kilns.copy().rename(columns = {'kiln_id':'id'})
emissions_gdf = emissions_gdf.to_crs(location_gdf.crs)

# Create a buffer around each kiln location (if not already done)
neighbor_distance = 10000  # e.g., 10km
location_gdf['buffer'] = location_gdf.geometry.buffer(neighbor_distance)

# Set the 'buffer' column as the active geometry column
location_gdf_buffer = location_gdf.set_geometry('buffer')

# Now perform the spatial join using the active geometry from location_gdf_buffer.
neighbors = gpd.sjoin(
    emissions_gdf, 
   location_gdf_buffer[['id', 'category','buffer']], 
    how='inner', 
    predicate='within'
)

neighbors['firing_start_date'] = pd.to_datetime(neighbors['firing_start_date'])
neighbors['firing_end_date'] = pd.to_datetime(neighbors['firing_end_date'])

# Define the emission columns for which to compute the mean
aggregations = {
    'avg_co2_tons_per_day': 'mean',
    'avg_co2_tons_per_month': 'mean',
    'avg_pm25_kg_per_day': 'mean',
    'avg_pm25_kg_per_month': 'mean',
    'firing_start_date': 'median',
    'firing_end_date': 'median'
}

neighbors_mean = neighbors.groupby('id').agg(aggregations).reset_index()

# Optionally, convert back to a string format (year-month-day)
neighbors_mean['firing_start_date'] = neighbors_mean['firing_start_date'].dt.strftime('%Y-%m-%d')
neighbors_mean['firing_end_date'] = neighbors_mean['firing_end_date'].dt.strftime('%Y-%m-%d')
neighbors_mean.head()

,id,avg_co2_tons_per_day,avg_co2_tons_per_month,avg_pm25_kg_per_day,avg_pm25_kg_per_month,firing_start_date,firing_end_date
0,102820,8.809577,264.287301,23.259977,697.799309,2022-11-07,2023-04-22
1,102829,8.809577,264.287301,23.259977,697.799309,2022-11-07,2023-04-22
2,102830,8.809577,264.287301,23.259977,697.799309,2022-11-07,2023-04-22
3,102833,8.809577,264.287301,23.259977,697.799309,2022-11-07,2023-04-22
4,102834,8.809577,264.287301,23.259977,697.799309,2022-11-07,2023-04-22


In [72]:
location_merged = pd.merge(location_gdf, neighbors_mean, on = 'id', how = 'left')

# For those kilns that do not have neighboring kilns in RCT/scaling data, impute missing emissions using whole average values
location_merged['avg_co2_tons_per_day'] = location_merged['avg_co2_tons_per_day'].fillna(
    emissions_df['avg_co2_tons_per_day'].mean()
)
location_merged['avg_co2_tons_per_month'] = location_merged['avg_co2_tons_per_month'].fillna(
    emissions_df['avg_co2_tons_per_month'].mean()
)
location_merged['avg_pm25_kg_per_day'] = location_merged['avg_pm25_kg_per_day'].fillna(
    emissions_df['avg_pm25_kg_per_day'].mean()
)
location_merged['avg_pm25_kg_per_month'] = location_merged['avg_pm25_kg_per_month'].fillna(
    emissions_df['avg_pm25_kg_per_month'].mean()
)

location_merged['firing_start_date'] = location_merged['firing_start_date'].fillna(
    emissions_df['firing_start_date'].median()
)

emissions_df['firing_end_date'] = pd.to_datetime(emissions_df['firing_end_date'])
location_merged['firing_end_date'] = pd.to_datetime(location_merged['firing_end_date'])

location_merged['firing_end_date'] = location_merged['firing_end_date'].fillna(
    emissions_df['firing_end_date'].median()
)

location_merged = location_merged.rename(columns = {'id':'kiln_id', 'lat':'latitude','long':'longitude'}).drop(columns = ['buffer'])
location_merged.head()

,kiln_id,category,division,geometry,avg_co2_tons_per_day,avg_co2_tons_per_month,avg_pm25_kg_per_day,avg_pm25_kg_per_month,firing_start_date,firing_end_date
0,100000,PNAS,Chittagong,POINT (5241276.392 3739925.248),10.498142,314.944254,27.718305,831.549157,2022-11-21 00:00:00,2023-05-01
1,100001,PNAS,Chittagong,POINT (5241189.938 3740082.634),10.498142,314.944254,27.718305,831.549157,2022-11-21 00:00:00,2023-05-01
2,100002,PNAS,Chittagong,POINT (5240070.054 3740287.121),10.498142,314.944254,27.718305,831.549157,2022-11-21 00:00:00,2023-05-01
3,100003,PNAS,Chittagong,POINT (5241021.955 3740369.603),10.498142,314.944254,27.718305,831.549157,2022-11-21 00:00:00,2023-05-01
4,100004,PNAS,Chittagong,POINT (5239764.288 3751000.298),10.498142,314.944254,27.718305,831.549157,2022-11-21 00:00:00,2023-05-01


In [84]:
fin_gdf = pd.concat([location_merged, emissions_df])
fin_df = fin_gdf.drop(columns = ['season_days', 'geometry','avg_co2_tons_per_day','avg_pm25_kg_per_day'])
fin_df['firing_start_date'] = pd.to_datetime(fin_df['firing_start_date'])
fin_df['firing_end_date'] = pd.to_datetime(fin_df['firing_end_date'])

fin_df.head()

,kiln_id,category,division,avg_co2_tons_per_month,avg_pm25_kg_per_month,firing_start_date,firing_end_date,latitude,longitude
0,100000,PNAS,Chittagong,314.944254,831.549157,2022-11-21,2023-05-01,NaN,NaN
1,100001,PNAS,Chittagong,314.944254,831.549157,2022-11-21,2023-05-01,NaN,NaN
2,100002,PNAS,Chittagong,314.944254,831.549157,2022-11-21,2023-05-01,NaN,NaN
3,100003,PNAS,Chittagong,314.944254,831.549157,2022-11-21,2023-05-01,NaN,NaN
4,100004,PNAS,Chittagong,314.944254,831.549157,2022-11-21,2023-05-01,NaN,NaN


In [86]:
fin_df['category'].unique()

array(['PNAS', 'RCT', 'Scaling'], dtype=object)

In [88]:
fin_df['firing_end_date'].max()

Timestamp('2023-07-24 00:00:00')

In [90]:
def month_active_fraction(row, current_month):
    # current_month is a Timestamp representing the first day of the month (e.g., 2022-11-01)
    start = row['firing_start_date']
    end = row['firing_end_date']
    
    # Define the month’s start and end dates.
    month_start = current_month
    last_day = calendar.monthrange(current_month.year, current_month.month)[1]
    month_end = current_month.replace(day=last_day)
    
    # Overlapping period:
    overlap_start = max(start, month_start)
    overlap_end = min(end, month_end)
    
    # If there's no overlap, return 0
    if overlap_end < overlap_start:
        return 0
    
    # +1 to count inclusive days.
    overlap_days = (overlap_end - overlap_start).days + 1
    total_days = last_day
    fraction = overlap_days / total_days
    return fraction
    

def expand_row(row):
    # Ensure firing dates are datetime objects (if not already)
    start_month = row['firing_start_date'].replace(day=1)
    end_month = row['firing_end_date'].replace(day=1)
    
    # Create a date range for the first day of each month in the firing period
    monthly_dates = pd.date_range(start=start_month, end=end_month, freq='MS')
    
    expanded = []
    for d in monthly_dates:
        new_row = row.copy()
        # Compute the fraction of the month that is active
        fraction = month_active_fraction(row, d)
        
        # Create a new column 'date' with year-month (formatted as 'YYYY-MM')
        new_row['date'] = d.strftime('%Y-%m')
        
        # Adjust emissions by the fraction of the month active.
        # For example, if only half the month was active, emissions are halved.
        new_row['avg_co2_tons_per_month'] = row['avg_co2_tons_per_month'] * fraction
        new_row['avg_pm25_kg_per_month'] = row['avg_pm25_kg_per_month'] * fraction
        expanded.append(new_row)
    return expanded

# Expand each row in fin_df
expanded_rows = []
for _, row in fin_df.iterrows():
    expanded_rows.extend(expand_row(row))

# Create the new DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Select only the desired columns
result_df = expanded_df[['latitude', 'longitude', 'kiln_id', 'category', 'date',
                           'avg_co2_tons_per_month', 'avg_pm25_kg_per_month']]

result_df.head()

,latitude,longitude,kiln_id,category,date,avg_co2_tons_per_month,avg_pm25_kg_per_month
0,NaN,NaN,100000,PNAS,2022-11,104.981418,277.183052
0,NaN,NaN,100000,PNAS,2022-12,314.944254,831.549157
0,NaN,NaN,100000,PNAS,2023-01,314.944254,831.549157
0,NaN,NaN,100000,PNAS,2023-02,314.944254,831.549157
0,NaN,NaN,100000,PNAS,2023-03,314.944254,831.549157


In [91]:
result_df['category'].unique()

array(['PNAS', 'RCT', 'Scaling'], dtype=object)

In [92]:
result_df = result_df[['kiln_id','category','date','avg_co2_tons_per_month','avg_pm25_kg_per_month']].copy()
result_df

,kiln_id,category,date,avg_co2_tons_per_month,avg_pm25_kg_per_month
0,100000,PNAS,2022-11,104.981418,277.183052
0,100000,PNAS,2022-12,314.944254,831.549157
0,100000,PNAS,2023-01,314.944254,831.549157
0,100000,PNAS,2023-02,314.944254,831.549157
0,100000,PNAS,2023-03,314.944254,831.549157
...,...,...,...,...,...
95,33122,Scaling,2023-01,229.314647,605.460806
95,33122,Scaling,2023-02,229.314647,605.460806
95,33122,Scaling,2023-03,229.314647,605.460806
95,33122,Scaling,2023-04,229.314647,605.460806


In [102]:
# put back the geometry
kilns_fin = pd.merge(result_df, kilns_gps[['kiln_id','geometry']].copy(), on = 'kiln_id', how = 'left')
kilns_fin

,kiln_id,category,date,avg_co2_tons_per_month,avg_pm25_kg_per_month,geometry
0,100000,PNAS,2022-11,104.981418,277.183052,POINT (5241276.392 3739925.248)
1,100000,PNAS,2022-12,314.944254,831.549157,POINT (5241276.392 3739925.248)
2,100000,PNAS,2023-01,314.944254,831.549157,POINT (5241276.392 3739925.248)
3,100000,PNAS,2023-02,314.944254,831.549157,POINT (5241276.392 3739925.248)
4,100000,PNAS,2023-03,314.944254,831.549157,POINT (5241276.392 3739925.248)
...,...,...,...,...,...,...
45303,33122,Scaling,2023-01,229.314647,605.460806,POINT (5029732.397 4047802.023)
45304,33122,Scaling,2023-02,229.314647,605.460806,POINT (5029732.397 4047802.023)
45305,33122,Scaling,2023-03,229.314647,605.460806,POINT (5029732.397 4047802.023)
45306,33122,Scaling,2023-04,229.314647,605.460806,POINT (5029732.397 4047802.023)


In [ ]:
result_df.to_csv(os.path.join(path, "co2_pm25_emissions_brik_kiln.csv"), index = False)